### WIKIPEDIA RETRIEVER

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retriever = WikipediaRetriever(top_k_results = 2, lang = "en")

In [3]:
query = "the geopolitical history of India and Pakistan from the perspective of an American "

docs = retriever.invoke(query)

In [4]:
docs

[Document(metadata={'title': 'Indo-Pakistani wars and conflicts', 'summary': 'Since the Partition of British India in 1947 and subsequent creation of the dominions of India and Pakistan, the two countries have been involved in a number of wars, conflicts, and military standoffs. A long-running dispute over Kashmir and cross-border terrorism have been the predominant cause of conflict between the two states, with the exception of the Indo-Pakistani War of 1971, which occurred as a direct result of hostilities stemming from the Bangladesh Liberation War in erstwhile East Pakistan (now Bangladesh).\n\n', 'source': 'https://en.wikipedia.org/wiki/Indo-Pakistani_wars_and_conflicts'}, page_content='Since the Partition of British India in 1947 and subsequent creation of the dominions of India and Pakistan, the two countries have been involved in a number of wars, conflicts, and military standoffs. A long-running dispute over Kashmir and cross-border terrorism have been the predominant cause of

In [5]:
for i, doc in enumerate(docs):
    print(f"\n--- Result{i+1}---")
    print(f"Content:\n{doc.page_content}...")


--- Result1---
Content:
Since the Partition of British India in 1947 and subsequent creation of the dominions of India and Pakistan, the two countries have been involved in a number of wars, conflicts, and military standoffs. A long-running dispute over Kashmir and cross-border terrorism have been the predominant cause of conflict between the two states, with the exception of the Indo-Pakistani War of 1971, which occurred as a direct result of hostilities stemming from the Bangladesh Liberation War in erstwhile East Pakistan (now Bangladesh).


== Background ==

The Partition of India came in 1947 with the sudden grant of independence. It was the intention of those who wished for a Muslim state to have a clean partition between independent and equal "Pakistan" and "Hindustan" once independence came.
Nearly one third of the Muslim population of India remained in the new India.
Inter-communal violence between Hindus, Sikhs and Muslims resulted in between 200,000 and 2 million casualties

### VECTOR STORE RETRIEVER

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [7]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)


C:\Users\dubey\AppData\Local\Temp\ipykernel_18772\2914580400.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\dubey\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
print(type(vectorstore))


<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [17]:
query = "what is Chroma used for?"
results = vectorstore.invoke(query)

In [19]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


### MMR

In [21]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [23]:
from langchain_community.vectorstores import FAISS


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [24]:
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [25]:
query = "What is langchain?"
results = retriever.invoke(query)

In [26]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
Embeddings are vector representations of text.


### Multiquery Retriever

In [27]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [28]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [29]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [30]:
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [32]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=GoogleGenerativeAI(model = 'gemini-1.5-pro')
)

In [33]:
query = "How to improve energy levels and maintain balance?"

In [ ]:
similarity_results = similarity_retriever.invoke(query)
## multiquery_results= multiquery_retriever.invoke(query)

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/gen

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions 

In [37]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

# for i, doc in enumerate(multiquery_results):
#     print(f"\n--- Result {i+1} ---")
#     print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 5 ---
Photosynthesis enables plants to produce energy by converting sunlight.
******************************************************************************************************************************************************
